In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("C:\\Users\\YASH\\Documents\\GitHub\\A-Decentralized-Framework-for-AI-based-Resource-Allocation-in-Open-RAN\\data\\final_ran_dataset.csv")
print(df.head())
print(df.shape)

   traffic_load  num_users  packet_count  avg_packet_size  allocated_bandwidth
0      0.017677      0.375           0.0         0.004402             0.017677
1      0.028851      0.625           0.0         0.000000             0.028851
2      0.044450      0.375           0.0         0.014374             0.044450
3      0.023949      0.750           0.0         0.006602             0.023949
4      0.258286      0.625           0.0         0.034113             0.258286
(785, 5)


In [3]:
X = df[['traffic_load', 'num_users', 'packet_count', 'avg_packet_size']].values
y = df['allocated_bandwidth'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [6]:
class CentralizedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        return self.net(x)

In [7]:
model = CentralizedModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [9]:
epochs = 30
loss_history = []

for epoch in range(epochs):
    epoch_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    loss_history.append(avg_loss)

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

Epoch [1/30], Loss: 0.0088
Epoch [2/30], Loss: 0.0055
Epoch [3/30], Loss: 0.0048
Epoch [4/30], Loss: 0.0048
Epoch [5/30], Loss: 0.0041
Epoch [6/30], Loss: 0.0038
Epoch [7/30], Loss: 0.0037
Epoch [8/30], Loss: 0.0034
Epoch [9/30], Loss: 0.0031
Epoch [10/30], Loss: 0.0029
Epoch [11/30], Loss: 0.0026
Epoch [12/30], Loss: 0.0024
Epoch [13/30], Loss: 0.0022
Epoch [14/30], Loss: 0.0019
Epoch [15/30], Loss: 0.0017
Epoch [16/30], Loss: 0.0015
Epoch [17/30], Loss: 0.0013
Epoch [18/30], Loss: 0.0011
Epoch [19/30], Loss: 0.0010
Epoch [20/30], Loss: 0.0008
Epoch [21/30], Loss: 0.0007
Epoch [22/30], Loss: 0.0006
Epoch [23/30], Loss: 0.0005
Epoch [24/30], Loss: 0.0004
Epoch [25/30], Loss: 0.0003
Epoch [26/30], Loss: 0.0003
Epoch [27/30], Loss: 0.0002
Epoch [28/30], Loss: 0.0002
Epoch [29/30], Loss: 0.0002
Epoch [30/30], Loss: 0.0002


In [10]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)

print("Test MSE:", test_loss.item())

Test MSE: 5.952213177806698e-05


In [11]:
torch.save(model.state_dict(), "centralized_results.pth")
print("Centralized baseline model saved")

Centralized baseline model saved
